In [4]:
import pandas as pd
import numpy as np
import datetime

In [5]:
df_flights = pd.read_csv('../../raw_data/flights.csv',nrows=500000)
df_airports = pd.read_csv('../../raw_data/airports.csv')
df_airlines = pd.read_csv('../../raw_data/airlines.csv')

In [6]:
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [7]:
df_flights['ARRIVAL_TIME'] = df_flights['ARRIVAL_TIME'].apply(format_heure)

In [8]:
df_flights['SCHEDULED_DEPARTURE'] = df_flights['SCHEDULED_DEPARTURE'].apply(format_heure)

In [9]:
df_flights['SCHEDULED_ARRIVAL'] = df_flights['SCHEDULED_ARRIVAL'].apply(format_heure)

In [10]:
df_flights['DEPARTURE_TIME'] = df_flights['DEPARTURE_TIME'].apply(format_heure)

In [11]:
df_flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,00:05:00,...,04:08:00,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,00:10:00,...,07:41:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,00:20:00,...,08:11:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,00:20:00,...,07:56:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,00:25:00,...,02:59:00,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2015,2,3,2,WN,740,N647SW,DAL,MCO,06:05:00,...,09:10:00,-20.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
499996,2015,2,3,2,WN,1711,N8315C,MDW,DEN,06:05:00,...,07:44:00,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
499997,2015,2,3,2,WN,371,N438WN,MDW,PIT,06:05:00,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
499998,2015,2,3,2,WN,1267,N211WN,MDW,TPA,06:05:00,...,09:27:00,-18.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_flights['DEPARTURE_DELAY']

0        -11.0
1         -8.0
2         -2.0
3         -5.0
4         -1.0
          ... 
499995    -7.0
499996     0.0
499997     NaN
499998     0.0
499999    -1.0
Name: DEPARTURE_DELAY, Length: 500000, dtype: float64

In [44]:
df_flights['ORIGIN_AIRPORT']!=object.value_counts()

AttributeError: type object 'object' has no attribute 'value_counts'